In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn

# 1 数据读入

In [2]:
data_path = './proceed/train_data_all.csv'
data=pd.read_csv(data_path,encoding='utf-8',sep='|')
data['content'].str.len().describe()

count    24992.000000
mean       419.802937
std        121.761374
min         11.000000
25%        330.000000
50%        512.000000
75%        512.000000
max        513.000000
Name: content, dtype: float64

## 构造训练集和测试集

In [11]:
df=sklearn.utils.shuffle(data) #随机打乱
train_data = df.sample(frac=0.9, random_state=0, axis=0)
dev_data = df.drop(train_data.index)
train_data_path = './proceed/train_data.csv'
dev_data_path = './proceed/dev_data.csv'
# #将训练数据写入到文件中
train_data.to_csv(train_data_path,  index=False,encoding='utf-8',sep ='|',header =['id','content','abstract'])
# #将测试数据写入文件中
dev_data.to_csv(dev_data_path,  index=False,encoding='utf-8',sep ='|',header =['id','content','abstract'])

In [12]:
print("================train===============")
print(train_data['content'].str.len().describe())
print("=================dev=================")
print(dev_data['content'].str.len().describe())
print("====================================")

================train===============
count    22492.000000
mean       419.649475
std        121.892133
min         11.000000
25%        330.000000
50%        512.000000
75%        512.000000
max        513.000000
Name: content, dtype: float64
=================dev=================
count    2500.000000
mean      421.183600
std       120.593983
min        18.000000
25%       328.000000
50%       512.000000
75%       512.000000
max       513.000000
Name: content, dtype: float64


In [13]:
from paddlenlp.datasets import load_dataset
# # q:上面的函数还能怎么写
# # a:可以用pandas读取csv文件，然后用iterrows()方法遍历每一行，然后yield
def read_from_csv(data_path):
    data=pd.read_csv(data_path,encoding='utf-8',sep='|')
    for index, row in data.iterrows():
        content, abstract = row['content'],row['abstract']
        yield {'content': content, 'abstract': abstract}

# data_path为read()方法的参数
train_dataset  = load_dataset(read_from_csv, data_path=train_data_path, lazy=False, split="train")
dev_dataset  = load_dataset(read_from_csv, data_path=dev_data_path, lazy=False, split="dev")
print(len(train_dataset))
print(len(dev_dataset))

D:\Program\Anaconda\envs\paddle\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


22501
2500


In [37]:
from paddlenlp.transformers import PegasusForConditionalGeneration, PegasusChineseTokenizer
from paddlenlp.transformers import LinearDecayWithWarmup
from paddle.io import BatchSampler, DistributedBatchSampler, DataLoader
from paddlenlp.data import DataCollatorForSeq2Seq

#创建Tokenizer，用于分词，将token映射成id。
1# 初始化分词器
tokenizer = PegasusChineseTokenizer.from_pretrained('IDEA-CCNL/Randeng-Pegasus-238M-Summary-Chinese')
# 初始化模型，'IDEA-CCNL/Randeng-Pegasus-238M-Summary-Chinese/'IDEA-CCNL/Randeng-Pegasus-538M-Summary-Chinese
model = PegasusForConditionalGeneration.from_pretrained('IDEA-CCNL/Randeng-Pegasus-238M-Summary-Chinese')

# tokenizer = PegasusChineseTokenizer.from_pretrained('PaddlePaddle/Randeng-Pegasus-238M-Summary-Chinese-SSTIA')
# # 初始化模型，'IDEA-CCNL/Randeng-Pegasus-238M-Summary-Chinese/'IDEA-CCNL/Randeng-Pegasus-538M-Summary-Chinese
# model = PegasusForConditionalGeneration.from_pretrained('PaddlePaddle/Randeng-Pegasus-238M-Summary-Chinese-SSTIA')
# 组装 Batch 数据 & Padding
batchify_fn = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

[2023-06-04 23:54:15,523] [    INFO] - Already cached /home/ubuntu/.paddlenlp/models/IDEA-CCNL/Randeng-Pegasus-238M-Summary-Chinese/vocab.txt
[2023-06-04 23:54:15,524] [    INFO] - Already cached /home/ubuntu/.paddlenlp/models/IDEA-CCNL/Randeng-Pegasus-238M-Summary-Chinese/added_tokens.json
[2023-06-04 23:54:15,525] [    INFO] - Already cached /home/ubuntu/.paddlenlp/models/IDEA-CCNL/Randeng-Pegasus-238M-Summary-Chinese/special_tokens_map.json
[2023-06-04 23:54:15,526] [    INFO] - Already cached /home/ubuntu/.paddlenlp/models/IDEA-CCNL/Randeng-Pegasus-238M-Summary-Chinese/tokenizer_config.json
[2023-06-04 23:54:15,584] [    INFO] - Already cached /home/ubuntu/.paddlenlp/models/IDEA-CCNL/Randeng-Pegasus-238M-Summary-Chinese/model_state.pdparams
[2023-06-04 23:54:15,585] [    INFO] - Already cached /home/ubuntu/.paddlenlp/models/IDEA-CCNL/Randeng-Pegasus-238M-Summary-Chinese/model_config.json


# 训练

In [38]:
#安装需要的相关库
import os
import json
import argparse
import random
import time
import paddle
import paddlenlp
import distutils.util
from pprint import pprint
from functools import partial
from tqdm import tqdm
import numpy as np
import math
from datasets import load_dataset
import contextlib
from rouge import Rouge
from visualdl import LogWriter


import pandas as pd 
from paddlenlp.datasets import MapDataset
import paddle.nn as nn
from paddlenlp.utils.log import logger
from paddlenlp.metrics import BLEU
from paddlenlp.data import Tuple, Pad

In [39]:
#定义convert_example，将content和title文本映射成int类型的id，同时构造labels。
def convert_example(example, text_column, summary_column, tokenizer,
                    max_source_length, max_target_length):
    """
    构造模型的输入.
    """
    inputs = example[text_column]
    targets = example[summary_column]
    # 分词
    model_inputs = tokenizer(inputs,
                             max_length=max_source_length,
                             padding=False,
                             truncation=True,
                             return_attention_mask=True)
    labels = tokenizer(targets,
                       max_length=max_target_length,
                       padding=False,
                       truncation=True)
    # 得到labels，后续通过DataCollatorForSeq2Seq进行移位
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [40]:
#由于预训练模型限制，这里把文本最大长度设置为512
# 文本的最大长度
max_source_length = 512
# 摘要的最大长度
max_target_length = 160
# 摘要的最小长度
min_target_length = 0

In [41]:
#使用partial函数指定默认参数，使用map函数转换数据。map函数把原来的文本根据词汇表的编号转换成了相应的id，为了便于理解，这里把训练集合的部分样本展示出来。
# 定义转换器
trans_func = partial(convert_example,
                     text_column='content',
                     summary_column='abstract',
                     tokenizer=tokenizer,
                     max_source_length=max_source_length,
                     max_target_length=max_target_length)
                     
# train_dataset和dev_dataset分别转换
train_dataset = train_dataset.map(trans_func,
                                  batched=False,
                                  lazy =False)

dev_dataset = dev_dataset.map(trans_func,
                              batched=False,
                            lazy =False)
# 输出训练集的前 2 条样本
for idx, example in enumerate(train_dataset):
    if idx < 2:
        print(example) 

{'input_ids': [1117, 131, 1266, 230, 21328, 4054, 13055, 1266, 13048, 1266, 221, 1909, 1857, 5226, 1608, 1909, 26198, 1909, 15944, 39771, 615, 6659, 36230, 791, 10961, 1266, 3399, 4813, 221, 12695, 27871, 1101, 6445, 12695, 1909, 2349, 2355, 791, 1266, 3399, 4659, 23162, 1266, 1608, 21933, 21018, 3399, 2274, 615, 197, 2334, 333, 6659, 36230, 791, 10961, 505, 1608, 1909, 26198, 1909, 10838, 2421, 230, 299, 4919, 2587, 2587, 275, 696, 1227, 13984, 30698, 4920, 2274, 18423, 1909, 25537, 38129, 23167, 1117, 131, 1477, 791, 14093, 1477, 1101, 709, 791, 14093, 5034, 7018, 200, 6800, 39223, 15203, 2274, 22617, 2334, 6683, 10753, 5034, 2274, 18817, 221, 44582, 1266, 3399, 1909, 1608, 21933, 3290, 15944, 1418, 131, 4656, 19164, 23548, 676, 8398, 1979, 435, 1454, 15214, 15214, 1909, 4919, 346, 18739, 23476, 696, 275, 791, 1266, 1909, 1608, 12268, 823, 4599, 1608, 21933, 3851, 1078, 3299, 1117, 131, 4503, 1909, 15944, 15203, 2274, 22617, 2334, 520, 696, 520, 2274, 823, 36566, 3399, 1909, 3441, 33

In [42]:
train_data_loader  = paddle.io.DataLoader(
    dataset=train_dataset,
    batch_size=8,
    return_list=True,
    collate_fn=batchify_fn)
dev_batch_sampler  = BatchSampler(
    dataset=dev_dataset,
    batch_size=8,
    shuffle=False)




# 构造测试Dataloader
dev_data_loader = DataLoader(dataset=dev_dataset,
                             batch_sampler=dev_batch_sampler,
                             num_workers=0,
                             collate_fn=batchify_fn,
                             return_list=True)

## 训练配置

In [45]:
# 学习率预热比例
warmup = 0.02
# 学习率
learning_rate = 5e-5
# 训练轮次
num_epochs =50
# 训练总步数
num_training_steps = len(train_data_loader) * num_epochs
# AdamW优化器参数epsilon
adam_epsilon = 1e-6
# AdamW优化器参数weight_decay
weight_decay=0.01
# 训练中，每100个log_steps打印一次日志
log_steps = 100
# 训练中，每隔eval_steps进行一次模型评估
eval_steps = 10000
# 使用SSTIA
model.use_SSTIA = True
model.mix_ratio = 0.3


# 训练模型保存路径
output_dir = 'checkpoints'
# 解码beam size
num_beams = 4

log_writer = LogWriter('visualdl_log_dir')
lr_scheduler = LinearDecayWithWarmup(learning_rate, num_training_steps, warmup)

# LayerNorm参数不参与weight_decay
decay_params = [
    p.name for n, p in model.named_parameters()
    if not any(nd in n for nd in ["bias", "norm"])
]

# 优化器AdamW
optimizer = paddle.optimizer.AdamW(
    learning_rate=lr_scheduler,
    beta1=0.9,
    beta2=0.999,
    epsilon=adam_epsilon,
    parameters=model.parameters(),
    weight_decay=weight_decay,
    apply_decay_param_fun=lambda x: x in decay_params)

In [46]:
print(num_training_steps)

156300


In [47]:
# 计算训练评估参数Rouge-1，Rouge-2，Rouge-L
def compute_metrics(preds, targets):
    assert len(preds) == len(targets), (
        'The length of pred_responses should be equal to the length of '
        'target_responses. But received {} and {}.'.format(
            len(preds), len(targets)))
    rouge = Rouge()
    
    scores = []
    for pred, target in zip(preds, targets):
        try:
            score = rouge.get_scores(' '.join(pred), ' '.join(target))
            scores.append([
                score[0]['rouge-1']['f'], score[0]['rouge-2']['f'],
                score[0]['rouge-l']['f']
            ])
        except ValueError:
            scores.append([0, 0, 0])
    rouge1 = np.mean([i[0] for i in scores])
    rouge2 = np.mean([i[1] for i in scores])
    rougel = np.mean([i[2] for i in scores])
    
    print('\n' + '*' * 15)
    print('The auto evaluation result is:')
    print('rouge-1:', round(rouge1*100, 2))
    print('rouge-2:', round(rouge2*100, 2))
    print('rouge-L:', round(rougel*100, 2))
   
    return rouge1, rouge2, rougel

In [16]:
# 模型评估函数
@paddle.no_grad()
def evaluate(model, data_loader, tokenizer, min_target_length,
             max_target_length):
    model.eval()
    all_preds = []
    all_labels = []
    model = model._layers if isinstance(model, paddle.DataParallel) else model
    for batch in tqdm(data_loader, total=len(data_loader), desc="Eval step"):
        labels = batch.pop('labels').numpy()
        # 模型生成
        preds = model.generate(input_ids=batch['input_ids'],
                               attention_mask=batch['attention_mask'],
                               min_length=min_target_length,
                               max_length=max_target_length,
                               decode_strategy='greedy_search',
                               use_cache=True)[0]
        # tokenizer将id转为string
        all_preds.extend(
            tokenizer.batch_decode(preds.numpy(),
                                   skip_special_tokens=True,
                                   clean_up_tokenization_spaces=False))
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        all_labels.extend(
            tokenizer.batch_decode(labels,
                                   skip_special_tokens=True,
                                   clean_up_tokenization_spaces=False))
    rouge1, rouge2, rougel= compute_metrics(all_preds, all_labels)
    model.train()
    return rouge1, rouge2, rougel

## 定义训练

In [52]:
def train(model, train_data_loader):
    eval_steps = 20000
    global_step = 0
    best_rougel = 0
    tic_train = time.time()
    for epoch in range(num_epochs):
        for step, batch in enumerate(train_data_loader):
            global_step += 1
            # 模型前向训练，计算loss
            lm_logits, _, loss = model(**batch)
            loss.backward()
            optimizer.step()
            lr_scheduler.step()
            optimizer.clear_grad()
            if global_step % log_steps == 0:
                logger.info(
                    "global step %d/%d, epoch: %d, batch: %d, rank_id: %s, loss: %f, lr: %.10f, speed: %.4f step/s"
                    % (global_step, num_training_steps, epoch, step,
                        paddle.distributed.get_rank(), loss, optimizer.get_lr(),
                        log_steps / (time.time() - tic_train)))
                log_writer.add_scalar("train_loss", loss.numpy(), global_step)
                tic_train = time.time()
#             if global_step % eval_steps == 0 or global_step == num_training_steps or global_step in [10,6000,10000]:
#                 tic_eval = time.time()
#                 rouge1, rouge2, rougel = evaluate(model, dev_data_loader, tokenizer,
#                             min_target_length, max_target_length)
#                 logger.info("eval done total : %s s" % (time.time() - tic_eval))
#                 log_writer.add_scalar("eval_rouge1", rouge1, global_step)
#                 log_writer.add_scalar("eval_rouge2", rouge2, global_step)
#                 log_writer.add_scalar("eval_rougel", rougel, global_step)
#                 if best_rougel < rougel:
#                     best_rougel = rougel
#                     if paddle.distributed.get_rank() == 0:
#                         if not os.path.exists(output_dir):
#                             os.makedirs(output_dir)
#                         # Need better way to get inner model of DataParallel
#                         model_to_save = model._layers if isinstance(
#                             model, paddle.DataParallel) else model
#                         model_to_save.save_pretrained(output_dir)
#                         tokenizer.save_pretrained(output_dir)

            if global_step % eval_steps == 0 or global_step == num_training_steps or global_step in [0,1000,140000,150000]:
                if paddle.distributed.get_rank() == 0:
                        if not os.path.exists(output_dir):
                            os.makedirs(output_dir)
                        # Need better way to get inner model of DataParallel
                        model_to_save = model._layers if isinstance(
                            model, paddle.DataParallel) else model
                        model_to_save.save_pretrained(output_dir)
                        tokenizer.save_pretrained(output_dir)

In [53]:
# 调用
# 模型训练
train(model, train_data_loader)

[2023-06-05 22:28:40,360] [    INFO] - global step 100/156300, epoch: 0, batch: 99, rank_id: 0, loss: 3.599521, lr: 0.0000018234, speed: 0.9260 step/s
[2023-06-05 22:30:33,192] [    INFO] - global step 200/156300, epoch: 0, batch: 199, rank_id: 0, loss: 2.158661, lr: 0.0000034229, speed: 0.8864 step/s
[2023-06-05 22:32:26,325] [    INFO] - global step 300/156300, epoch: 0, batch: 299, rank_id: 0, loss: 2.505717, lr: 0.0000050224, speed: 0.8840 step/s
[2023-06-05 22:34:18,029] [    INFO] - global step 400/156300, epoch: 0, batch: 399, rank_id: 0, loss: 2.098660, lr: 0.0000066219, speed: 0.8954 step/s
[2023-06-05 22:36:04,628] [    INFO] - global step 500/156300, epoch: 0, batch: 499, rank_id: 0, loss: 2.317709, lr: 0.0000082214, speed: 0.9382 step/s
[2023-06-05 22:37:55,498] [    INFO] - global step 600/156300, epoch: 0, batch: 599, rank_id: 0, loss: 1.524417, lr: 0.0000098209, speed: 0.9021 step/s
[2023-06-05 22:39:45,066] [    INFO] - global step 700/156300, epoch: 0, batch: 699, rank

[2023-06-05 23:57:17,868] [    INFO] - global step 4900/156300, epoch: 1, batch: 1773, rank_id: 0, loss: 1.378705, lr: 0.0000494164, speed: 0.8926 step/s
[2023-06-05 23:59:07,226] [    INFO] - global step 5000/156300, epoch: 1, batch: 1873, rank_id: 0, loss: 1.137408, lr: 0.0000493837, speed: 0.9146 step/s
[2023-06-06 00:00:59,741] [    INFO] - global step 5100/156300, epoch: 1, batch: 1973, rank_id: 0, loss: 0.984272, lr: 0.0000493511, speed: 0.8889 step/s
[2023-06-06 00:02:48,769] [    INFO] - global step 5200/156300, epoch: 1, batch: 2073, rank_id: 0, loss: 1.023719, lr: 0.0000493184, speed: 0.9173 step/s
[2023-06-06 00:04:34,926] [    INFO] - global step 5300/156300, epoch: 1, batch: 2173, rank_id: 0, loss: 0.841073, lr: 0.0000492858, speed: 0.9421 step/s
[2023-06-06 00:06:26,500] [    INFO] - global step 5400/156300, epoch: 1, batch: 2273, rank_id: 0, loss: 0.502050, lr: 0.0000492531, speed: 0.8964 step/s
[2023-06-06 00:08:17,217] [    INFO] - global step 5500/156300, epoch: 1, ba

[2023-06-06 01:29:35,892] [    INFO] - global step 9900/156300, epoch: 3, batch: 521, rank_id: 0, loss: 0.697300, lr: 0.0000477842, speed: 0.9313 step/s
[2023-06-06 01:31:28,004] [    INFO] - global step 10000/156300, epoch: 3, batch: 621, rank_id: 0, loss: 0.603186, lr: 0.0000477516, speed: 0.8921 step/s
[2023-06-06 01:33:17,832] [    INFO] - global step 10100/156300, epoch: 3, batch: 721, rank_id: 0, loss: 0.652068, lr: 0.0000477189, speed: 0.9106 step/s
[2023-06-06 01:35:08,032] [    INFO] - global step 10200/156300, epoch: 3, batch: 821, rank_id: 0, loss: 0.954892, lr: 0.0000476863, speed: 0.9076 step/s
[2023-06-06 01:37:00,951] [    INFO] - global step 10300/156300, epoch: 3, batch: 921, rank_id: 0, loss: 0.937043, lr: 0.0000476536, speed: 0.8857 step/s
[2023-06-06 01:38:51,690] [    INFO] - global step 10400/156300, epoch: 3, batch: 1021, rank_id: 0, loss: 0.705247, lr: 0.0000476210, speed: 0.9032 step/s
[2023-06-06 01:40:41,634] [    INFO] - global step 10500/156300, epoch: 3, b

[2023-06-06 03:00:00,514] [    INFO] - global step 14800/156300, epoch: 4, batch: 2295, rank_id: 0, loss: 0.744137, lr: 0.0000461847, speed: 0.9181 step/s
[2023-06-06 03:01:51,690] [    INFO] - global step 14900/156300, epoch: 4, batch: 2395, rank_id: 0, loss: 0.836293, lr: 0.0000461521, speed: 0.8996 step/s
[2023-06-06 03:03:44,465] [    INFO] - global step 15000/156300, epoch: 4, batch: 2495, rank_id: 0, loss: 0.738998, lr: 0.0000461194, speed: 0.8868 step/s
[2023-06-06 03:05:40,502] [    INFO] - global step 15100/156300, epoch: 4, batch: 2595, rank_id: 0, loss: 0.621049, lr: 0.0000460868, speed: 0.8619 step/s
[2023-06-06 03:07:33,484] [    INFO] - global step 15200/156300, epoch: 4, batch: 2695, rank_id: 0, loss: 0.637083, lr: 0.0000460542, speed: 0.8852 step/s
[2023-06-06 03:09:23,591] [    INFO] - global step 15300/156300, epoch: 4, batch: 2795, rank_id: 0, loss: 0.609382, lr: 0.0000460215, speed: 0.9083 step/s
[2023-06-06 03:11:14,962] [    INFO] - global step 15400/156300, epoch

[2023-06-06 04:30:39,141] [    INFO] - global step 19700/156300, epoch: 6, batch: 943, rank_id: 0, loss: 0.382402, lr: 0.0000445852, speed: 0.9047 step/s
[2023-06-06 04:32:30,394] [    INFO] - global step 19800/156300, epoch: 6, batch: 1043, rank_id: 0, loss: 0.619550, lr: 0.0000445526, speed: 0.8990 step/s
[2023-06-06 04:34:21,416] [    INFO] - global step 19900/156300, epoch: 6, batch: 1143, rank_id: 0, loss: 0.449046, lr: 0.0000445200, speed: 0.9008 step/s
[2023-06-06 04:36:15,945] [    INFO] - global step 20000/156300, epoch: 6, batch: 1243, rank_id: 0, loss: 0.468883, lr: 0.0000444873, speed: 0.8733 step/s
[2023-06-06 04:36:26,892] [    INFO] - tokenizer config file saved in checkpoints/tokenizer_config.json
[2023-06-06 04:36:26,894] [    INFO] - Special tokens file saved in checkpoints/special_tokens_map.json
[2023-06-06 04:38:18,608] [    INFO] - global step 20100/156300, epoch: 6, batch: 1343, rank_id: 0, loss: 0.685623, lr: 0.0000444547, speed: 0.8153 step/s
[2023-06-06 04:40:

...

[2023-06-07 16:30:25,184] [    INFO] - global step 136600/156300, epoch: 43, batch: 2181, rank_id: 0, loss: 0.004327, lr: 0.0000064260, speed: 0.9290 step/s
[2023-06-07 16:32:16,148] [    INFO] - global step 136700/156300, epoch: 43, batch: 2281, rank_id: 0, loss: 0.005938, lr: 0.0000063934, speed: 0.9013 step/s
[2023-06-07 16:34:05,582] [    INFO] - global step 136800/156300, epoch: 43, batch: 2381, rank_id: 0, loss: 0.014332, lr: 0.0000063607, speed: 0.9139 step/s
[2023-06-07 16:35:56,243] [    INFO] - global step 136900/156300, epoch: 43, batch: 2481, rank_id: 0, loss: 0.002355, lr: 0.0000063281, speed: 0.9038 step/s
[2023-06-07 16:37:51,840] [    INFO] - global step 137000/156300, epoch: 43, batch: 2581, rank_id: 0, loss: 0.001638, lr: 0.0000062955, speed: 0.8652 step/s
[2023-06-07 16:39:47,851] [    INFO] - global step 137100/156300, epoch: 43, batch: 2681, rank_id: 0, loss: 0.005080, lr: 0.0000062628, speed: 0.8621 step/s
[2023-06-07 16:41:36,572] [    INFO] - global step 137200/

[2023-06-07 17:57:25,409] [    INFO] - global step 141300/156300, epoch: 45, batch: 629, rank_id: 0, loss: 0.001587, lr: 0.0000048918, speed: 0.9050 step/s
[2023-06-07 17:59:15,968] [    INFO] - global step 141400/156300, epoch: 45, batch: 729, rank_id: 0, loss: 0.004543, lr: 0.0000048592, speed: 0.9046 step/s
[2023-06-07 18:01:05,335] [    INFO] - global step 141500/156300, epoch: 45, batch: 829, rank_id: 0, loss: 0.001415, lr: 0.0000048265, speed: 0.9145 step/s
[2023-06-07 18:02:58,254] [    INFO] - global step 141600/156300, epoch: 45, batch: 929, rank_id: 0, loss: 0.002481, lr: 0.0000047939, speed: 0.8857 step/s
[2023-06-07 18:04:48,997] [    INFO] - global step 141700/156300, epoch: 45, batch: 1029, rank_id: 0, loss: 0.005140, lr: 0.0000047613, speed: 0.9031 step/s
[2023-06-07 18:06:39,166] [    INFO] - global step 141800/156300, epoch: 45, batch: 1129, rank_id: 0, loss: 0.003225, lr: 0.0000047286, speed: 0.9078 step/s
[2023-06-07 18:08:33,459] [    INFO] - global step 141900/1563

[2023-06-07 19:27:47,160] [    INFO] - global step 146200/156300, epoch: 46, batch: 2403, rank_id: 0, loss: 0.004024, lr: 0.0000032923, speed: 0.9022 step/s
[2023-06-07 19:29:40,313] [    INFO] - global step 146300/156300, epoch: 46, batch: 2503, rank_id: 0, loss: 0.001606, lr: 0.0000032597, speed: 0.8839 step/s
[2023-06-07 19:31:36,419] [    INFO] - global step 146400/156300, epoch: 46, batch: 2603, rank_id: 0, loss: 0.003898, lr: 0.0000032270, speed: 0.8614 step/s
[2023-06-07 19:33:28,158] [    INFO] - global step 146500/156300, epoch: 46, batch: 2703, rank_id: 0, loss: 0.004210, lr: 0.0000031944, speed: 0.8951 step/s
[2023-06-07 19:35:19,049] [    INFO] - global step 146600/156300, epoch: 46, batch: 2803, rank_id: 0, loss: 0.006353, lr: 0.0000031618, speed: 0.9019 step/s
[2023-06-07 19:37:10,088] [    INFO] - global step 146700/156300, epoch: 46, batch: 2903, rank_id: 0, loss: 0.002829, lr: 0.0000031291, speed: 0.9007 step/s
[2023-06-07 19:39:01,059] [    INFO] - global step 146800/

[2023-06-07 20:54:54,465] [    INFO] - global step 150900/156300, epoch: 48, batch: 851, rank_id: 0, loss: 0.001876, lr: 0.0000017581, speed: 0.8914 step/s
[2023-06-07 20:56:44,618] [    INFO] - global step 151000/156300, epoch: 48, batch: 951, rank_id: 0, loss: 0.007644, lr: 0.0000017255, speed: 0.9080 step/s
[2023-06-07 20:58:36,792] [    INFO] - global step 151100/156300, epoch: 48, batch: 1051, rank_id: 0, loss: 0.005407, lr: 0.0000016928, speed: 0.8916 step/s
[2023-06-07 21:00:28,134] [    INFO] - global step 151200/156300, epoch: 48, batch: 1151, rank_id: 0, loss: 0.011273, lr: 0.0000016602, speed: 0.8983 step/s
[2023-06-07 21:02:22,348] [    INFO] - global step 151300/156300, epoch: 48, batch: 1251, rank_id: 0, loss: 0.001919, lr: 0.0000016276, speed: 0.8757 step/s
[2023-06-07 21:04:12,508] [    INFO] - global step 151400/156300, epoch: 48, batch: 1351, rank_id: 0, loss: 0.002378, lr: 0.0000015949, speed: 0.9079 step/s
[2023-06-07 21:06:00,817] [    INFO] - global step 151500/15

[2023-06-07 22:25:26,612] [    INFO] - global step 155800/156300, epoch: 49, batch: 2625, rank_id: 0, loss: 0.001987, lr: 0.0000001586, speed: 0.8503 step/s
[2023-06-07 22:27:15,174] [    INFO] - global step 155900/156300, epoch: 49, batch: 2725, rank_id: 0, loss: 0.001197, lr: 0.0000001260, speed: 0.9213 step/s
[2023-06-07 22:29:08,297] [    INFO] - global step 156000/156300, epoch: 49, batch: 2825, rank_id: 0, loss: 0.002396, lr: 0.0000000934, speed: 0.8841 step/s
[2023-06-07 22:30:56,250] [    INFO] - global step 156100/156300, epoch: 49, batch: 2925, rank_id: 0, loss: 0.001298, lr: 0.0000000607, speed: 0.9265 step/s
[2023-06-07 22:32:48,262] [    INFO] - global step 156200/156300, epoch: 49, batch: 3025, rank_id: 0, loss: 0.001472, lr: 0.0000000281, speed: 0.8929 step/s
[2023-06-07 22:34:37,264] [    INFO] - global step 156300/156300, epoch: 49, batch: 3125, rank_id: 0, loss: 0.000058, lr: 0.0000000000, speed: 0.9175 step/s
[2023-06-07 22:34:48,029] [    INFO] - tokenizer config fi

# 输出测试

In [54]:
model = PegasusForConditionalGeneration.from_pretrained('checkpoints')
model.eval()
tokenizer = PegasusChineseTokenizer.from_pretrained('checkpoints')

In [55]:
num_beams = 4
def infer(text, model, tokenizer):
    tokenized = tokenizer(text, 
                          truncation=True, 
                          max_length=max_source_length, 
                          return_tensors='pd')
    preds, _ = model.generate(input_ids=tokenized['input_ids'],
                              max_length=160,
                              min_length=min_target_length,
                              decode_strategy='beam_search',
                              num_beams=num_beams)
    return(tokenizer.decode(preds[0], skip_special_tokens=True, clean_up_tokenization_spaces=False))

In [56]:
validation_file='./process_data_62/test_data.csv'
validation_data=pd.read_csv(validation_file,encoding='utf-8',sep='|',header=0)
text=validation_data.iat[0,1]
validation_data.head(5)

,id,content
0,25001,客:个我刚收到们个话费通知的短信我看说我上个月消费二十一块多吧有五元是我的包月费剩下有16块...
1,25002,客:个先说个不太好意思今天不是我之前投诉不是个扣费的问题，个今天今天应该给我反馈回来有服务电...
2,25003,客:个我套餐到期没人通知我原来是138块6的现在它我这刚发现现在是按198坐:帮查来电手机号...
3,25004,客:我想问我个话费现在欠坐:来电这号码欠费钱是坐:行号码现在欠费2月份的165.75165....
4,25005,客:你帮我查我有业务是返返钱的业务一放1月份的给我返个九十九十九月好像还能返28你帮我看客:...


In [57]:
validation_data['content'].str.len().describe()

count    7522.000000
mean      421.702340
std       120.174324
min        15.000000
25%       334.000000
50%       512.000000
75%       512.000000
max       513.000000
Name: content, dtype: float64

In [58]:
def read_file(filename,num=2):
    lines = []
    with open(filename, 'r', encoding='utf-8') as f:
        next(f)
        if num==3:
            for line in f:
                lines.append({"id":line.split("|")[0].strip(),"content":line.split("|")[1].strip(),"abstract":line.split("|")[2].strip()})
        else:
            for line in f:
                lines.append({"id":line.split("|")[0].strip(),"content":line.split("|")[1].strip()})
    return lines
infer(text, model, tokenizer)

'用户来电对01月产生的15.93元的流量费有疑义，我方经系统查询安上网日志向用户解释，用户不认可，我方按快速处理向用户核减费用，用户不认可要求核实原因，请处理，谢谢。'

In [59]:
validation_lines=read_file(validation_file)

In [60]:
validation_lines[0]

{'id': '25001',
 'content': '客:个我刚收到们个话费通知的短信我看说我上个月消费二十一块多吧有五元是我的包月费剩下有16块多的个的消费我没有进行过何的上网通话的我想查查个什费用坐:我看1月二十一块8上网流量费15块9毛3国内通话费0.15元到达电话打一分钟流量应该是超出一部分用15块9毛3的流量费客:对我肯定没有用流量我是退休在家我家里有WIFI我不出去玩一星之前我刚激活根本连买菜都不用我出去流量是怎出来的钱这肯定有积分问题坐:用五十五十九兆流量扣19块15块9毛7的费用客:59兆流量六块钱坐:电话对流量超出不认可给进行上报反馈后期会有专人给回电处理客:他最后给我回电怎处理，我要求这返返费我肯定没消费说电话电话有过，电话我没打过我家里人没用我电话打过每个人都有手续坐:现在无法给进行核实只能看到使用的超出是没有使用我给记录上报后期会有专人给客:你给我上报余额实际话费反正就这15块几吧我肯定没花点钱我肯定要交的我绝对没用过我就没出过门激活坐:好我给记录上报后期会有专人给回复保持电话畅通客:直接就给我发短信坐:24小时你回复电话坐:感谢来电稍后按两个一谢谢'}

In [61]:
result_data=pd.DataFrame()
idx = 0
for line in validation_lines:
    idx +=1
    validation_id=line['id']
    content=line['content']
    ret=infer(content,model, tokenizer)
    result={"id":validation_id,"ret":ret}
    result_data=result_data._append(result,ignore_index=True)
    if idx % 100 == 0 or idx in [1,2,3,4,5,6]:
        print("数据生成进行中，当前进行ID：",validation_id)
result_data.to_csv('result.csv',  index=False,encoding='utf-8',sep ='|',header =['id','ret'])

数据生成进行中，当前进行ID： 25001
数据生成进行中，当前进行ID： 25002
数据生成进行中，当前进行ID： 25003
数据生成进行中，当前进行ID： 25004
数据生成进行中，当前进行ID： 25005
数据生成进行中，当前进行ID： 25006
数据生成进行中，当前进行ID： 25100
数据生成进行中，当前进行ID： 25200
数据生成进行中，当前进行ID： 25300
数据生成进行中，当前进行ID： 25400
数据生成进行中，当前进行ID： 25500
数据生成进行中，当前进行ID： 25600
数据生成进行中，当前进行ID： 25700
数据生成进行中，当前进行ID： 25800
数据生成进行中，当前进行ID： 25900
数据生成进行中，当前进行ID： 26000
数据生成进行中，当前进行ID： 26100
数据生成进行中，当前进行ID： 26200
数据生成进行中，当前进行ID： 26300
数据生成进行中，当前进行ID： 26400
数据生成进行中，当前进行ID： 26500
数据生成进行中，当前进行ID： 26600
数据生成进行中，当前进行ID： 26700
数据生成进行中，当前进行ID： 26800
数据生成进行中，当前进行ID： 26900
数据生成进行中，当前进行ID： 27000
数据生成进行中，当前进行ID： 27100
数据生成进行中，当前进行ID： 27200
数据生成进行中，当前进行ID： 27300
数据生成进行中，当前进行ID： 27400
数据生成进行中，当前进行ID： 27500
数据生成进行中，当前进行ID： 27600
数据生成进行中，当前进行ID： 27700
数据生成进行中，当前进行ID： 27800
数据生成进行中，当前进行ID： 27900
数据生成进行中，当前进行ID： 28000
数据生成进行中，当前进行ID： 28100
数据生成进行中，当前进行ID： 28200
数据生成进行中，当前进行ID： 28300
数据生成进行中，当前进行ID： 28400
数据生成进行中，当前进行ID： 28500
数据生成进行中，当前进行ID： 28600
数据生成进行中，当前进行ID： 28700
数据生成进行中，当前进行ID： 28800
数据生成进行中，当前进行ID： 28900
数据生成进行中，当前

In [62]:
!tar -cvf checkpoints64.tar checkpoints/

checkpoints/
checkpoints/model_config.json
checkpoints/vocab.txt
checkpoints/model_state.pdparams
checkpoints/tokenizer_config.json
checkpoints/special_tokens_map.json


In [51]:
!rm -rf checkpoints/